In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os
from imutils import paths

# Load images
path_dir = r"C:\Hope_Learning\Deep_learning\Supervised_learning\Face_Mask_Detection"
print("Loading Images From File Directory...")
imgs_path = list(paths.list_images(path_dir))

data = []
labels = []

# Preprocess images and labels
for img_path in imgs_path:
    label = img_path.split(os.path.sep)[-2]  # Get label from the directory name
    labels.append(label)

    image = load_img(img_path, target_size=(128, 128))
    image = img_to_array(image)
    image = preprocess_input(image)  # Preprocess for MobileNetV2
    data.append(image)

# Convert to numpy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

# Label encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)

# Load the MobileNetV2 model
pre_trained_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(128, 128, 3)))

# Add new layers on top of the pre-trained model
join_our_model = pre_trained_model.output
join_our_model = AveragePooling2D(pool_size=(2, 2))(join_our_model)
join_our_model = Flatten(name="flatten")(join_our_model)
join_our_model = Dense(128, activation="relu")(join_our_model)
join_our_model = Dropout(0.5)(join_our_model)
join_our_model = Dense(2, activation="softmax")(join_our_model)

# Create the final model
model = Model(inputs=pre_trained_model.input, outputs=join_our_model)

# Freeze the layers of MobileNetV2
for layer in pre_trained_model.layers:
    layer.trainable = False

# Compile model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Train model
history = model.fit(aug.flow(X_train, y_train, batch_size=100),
                    steps_per_epoch=len(X_train) // 100,
                    validation_data=(X_test, y_test),
                    epochs=10)

# Save model
model.save('mask_detection_model.h5')

# Optional: Model evaluation
y_pred = model.predict(X_test, batch_size=32)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Print classification report
print(classification_report(y_true_classes, y_pred_classes, target_names=lb.classes_))


